In [1]:
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

In [2]:
load_dotenv()
## load the GROQ API Key
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")

model=ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it")

In [5]:
from langchain_core.messages import HumanMessage
result=model.invoke([HumanMessage(content="Hi , My name is Pavan and I am a Gen AI data scientist")])

In [ ]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
output=parser.invoke(result)
output


"Hello Pavan, it's nice to meet you!\n\nBeing a Gen AI data scientist is an exciting field to be in. What kind of work are you doing these days? Are you working on any interesting projects you can tell me about?\n\nI'm always eager to learn more about how people are using AI technology.\n"

In [8]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Pavan and I am a Gen AI data scientist"),
        AIMessage(content=output),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You said your name is Pavan and that you are a Gen AI data scientist.  \n\nIs there anything else you'd like to tell me about yourself or your work? I'm happy to chat! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 114, 'total_tokens': 162, 'completion_time': 0.087272727, 'prompt_time': 0.003580255, 'queue_time': 0.011774005, 'total_time': 0.090852982}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-10e39888-b1e9-4ef2-b1cf-93d090751b6e-0', usage_metadata={'input_tokens': 114, 'output_tokens': 48, 'total_tokens': 162})

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Pavan and I am a Gen AI data scientist")],
    config=config
)

In [13]:
response.content

"Hi Pavan, it's great to meet you! That's a really exciting field to be in. \n\nWhat kind of projects are you working on with Gen AI? Are you focused on a particular area like text generation, image synthesis, or something else entirely?  I'm always curious to hear about the innovative work people are doing with large language models.\n"

In [14]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Pavan.  😊  \n\nI remember that from our earlier conversation.  How can I help you today? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 202, 'total_tokens': 232, 'completion_time': 0.054545455, 'prompt_time': 0.008031757, 'queue_time': -0.008000197, 'total_time': 0.062577212}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-908927bc-a22c-4729-be25-7678ff1880af-0', usage_metadata={'input_tokens': 202, 'output_tokens': 30, 'total_tokens': 232})

In [15]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to use it!\n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John! It's nice to meet you.\n\nWhat can I do for you today?\n"

In [17]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John.  \n\nI remember you told me earlier! 😊  Is there anything else I can help you with? \n'

In [18]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Answer all the question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [19]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Pavan")]})

AIMessage(content="Hello Pavan! It's nice to meet you.  \n\nI'm ready to answer your questions. What can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 31, 'total_tokens': 68, 'completion_time': 0.067272727, 'prompt_time': 0.000435678, 'queue_time': 0.014676652, 'total_time': 0.067708405}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-9bcab774-aecc-420e-b730-140247840409-0', usage_metadata={'input_tokens': 31, 'output_tokens': 37, 'total_tokens': 68})

In [22]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [23]:
config2 = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Pavan")],
    config=config2
)

response

AIMessage(content="Hi Pavan, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😊 \n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 82, 'total_tokens': 110, 'completion_time': 0.050909091, 'prompt_time': 0.00321375, 'queue_time': 0.01197995, 'total_time': 0.054122841}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-26cb19da-b98d-47e9-96d2-13439cd450f2-0', usage_metadata={'input_tokens': 82, 'output_tokens': 28, 'total_tokens': 110})

In [24]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config2,
)

response.content

'Your name is Pavan. 😊  I remember!  \n\nIs there anything else I can help you with?  \n'

In [25]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [27]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Pavan")],"language":"Hindi"})
response.content

'नमस्ते पवन! \n\nमुझे खुशी है कि आप मुझसे जुड़े हैं। \n\nआपको क्या मदद चाहिए?  \n\n'

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config3 = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Pavan")],"language":"Hindi"},
    config=config3
)
repsonse.content

'नमस्ते पवन! 👋 \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आप क्या पूछना चाहते हैं? 😊 \n'

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [31]:
response.content

'आपका नाम Pavan है।  \n\nमैंने पहले ही आपसे बात की थी और आपने अपना नाम बताया था। \n'

In [32]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

c:\Users\pavan\Documents\Krish_GenAI\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\pavan\Documents\Krish_GenAI\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\pavan\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer 

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [34]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal information, including your ice cream preferences.  \n\nWhat's your favorite flavor?  🍦😊\n"

In [35]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2". 😊  \n\n\n\n'

In [36]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config4={"configurable":{"session_id":"chat5"}}

In [37]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config4,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n"